In [ ]:
import pandas as pd, numpy as np, time, math
import matplotlib.font_manager, matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, acf
import nilmtk
nilmtk.Appliance.allow_synonyms=False
from datetime import datetime, timedelta
from tslearn.metrics import soft_dtw, dtw, dtw_path
import seaborn as sns


In [ ]:
#get STD of appliance consumption and the number of activations
dataset = nilmtk.DataSet('REFIT.h5')
dataset.set_window('2014-06-20', '2014-09-20')
BUILD, LOAD = 16, "television"
LIST_APP = dataset.buildings[BUILD].elec
LOAD_std =  next(LIST_APP[LOAD].load(sample_period = 60)).values.flatten()
LOAD_act =  LIST_APP[LOAD].activation_series()


# Case study 1


## Similarity of appliance usage patterns

In [ ]:
#standard deviation of appliance consumption and number of activations
[np.std(LOAD_std), len(LOAD_act)]

#activation duration
#standard deviation, range, median, mean of activation durations
actv_dur = [len(LOAD_act[i]) for i in range(0, len(LOAD_act))]
[np.std(actv_dur), np.max(actv_dur) - np.min(actv_dur), np.median(actv_dur), np.mean(actv_dur)]

#get time between appliance activations
LOAD_time_bet_act = [(LOAD_act[i+1].index[0] - LOAD_act[i].index[-1]).total_seconds() for i in range(0, len(LOAD_act)-1)]

#mean activation time in seconds and hours
np.mean(LOAD_time_bet_act) #in seconds
np.mean(LOAD_time_bet_act)/3600 #in hours


In [ ]:
#from the five selected appliances, obtain the appliances that exist within the building 
#selected appliances
app_sel= ["television", "microwave", "kettle", "washing machine", "dish washer"]
#appliances within building
app_build = [LIST_APP.appliances[i].type['type'] for i in range(0, len(LIST_APP.appliances))]
#selected appliances within building
app_sel_build = [app_sel[j] for j in range(0, len(app_sel)) if app_sel[j] in app_build]
app_sel_build


In [ ]:
dic_LOAD, dic_acf = {}, {}
for j in app_sel_build:  
    dat = next(dataset.buildings[BUILD].elec[j].load(sample_period = 60))['2014-06-20' : '2014-09-20'].values.flatten()
    dat = [x for x in dat if str(x) != 'nan']
    if np.max(dat) > 0:
        dic_LOAD[j] = dat
        _acf = acf(dic_LOAD[j], nlags = len(dic_LOAD[j]) - 1, alpha = 0.05, qstat = True, fft = True)
        dic_acf["acf"+j] = _acf
        

In [ ]:
plt.rcParams['figure.figsize'] = [8, 8]
#choose the appliance to calculate the ACF
app = "acftelevision"
#ACF plot
plt.plot(dic_acf[app][0])
plt.title("ACF plot for the " + app.split("f")[1] + " in house " + str(BUILD), fontsize = 13)
plt.xlabel("Lag", fontsize = 12) 
plt.xticks(fontsize = 11)
plt.ylabel("Autocorrelation coefficient", fontsize = 12)
plt.yticks(fontsize = 11)


In [ ]:
#constructing the similarity matrix
DTW_matrix = np.zeros((len(dic_acf), len(dic_acf)))
for j in range(0, len(app_sel_build)):
    start = time.time()
    for i in range(0, len(dic_acf)):
        if i < j:
            KEYS = list(dic_acf.keys())
            if KEYS[i] in dic_acf: 
                if dic_acf[KEYS[i]] == []:
                    print("A" + KEYS[i] + "tem ACF list vazia")
                else:     
                    DTW_matrix[i, j] = (dtw(dic_acf[KEYS[j]][0][0 : math.floor(len(dic_acf[KEYS[j]][0])/3)], dic_acf[KEYS[i]][0][0 : math.floor(len(dic_acf[KEYS[i]][0])/3)]) +
                            dtw(dic_acf[KEYS[j]][0][math.floor(len(dic_acf[KEYS[j]][0])/3) + 1 : math.floor((2*len(dic_acf[KEYS[j]][0])/3))], dic_acf[KEYS[i]][0][math.floor(len(dic_acf[KEYS[i]][0])/3) + 1 : math.floor((2*len(dic_acf[KEYS[i]][0])/3))]) +
                            dtw(dic_acf[KEYS[j]][0][math.floor(2*len(dic_acf[KEYS[j]][0])/3) + 1 : (len(dic_acf[KEYS[j]][0]))], dic_acf[KEYS[i]][0][math.floor(2*len(dic_acf[KEYS[i]][0])/3) + 1 : len(dic_acf[KEYS[i]][0])]))/3
                    DTW_matrix[i, j] = round(DTW_matrix[i, j], 2)
                    DTW_matrix[j, i] = DTW_matrix[i, j]
    end = time.time()
    print(end - start)

DTW_matrix


In [ ]:
#get the median, standard deviation and range of the similarity of usage patterns for each appliance

list_aux1 = []
for i in range(0, len(DTW_matrix[0])):
    DTW_matrix_aux = [DTW_matrix[i][j] for j in range(0, len(DTW_matrix[i])) if DTW_matrix[i][j] != 0]
    list_aux1.append([round(np.median(DTW_matrix_aux), 2), round(np.std(DTW_matrix_aux), 2), round(max(DTW_matrix_aux) - min(DTW_matrix_aux), 2)])
    
list_aux1






# Case study 2


## Similarity of power profiles


In [ ]:
#auxiliary functions for running the similarity of power profiles

#rescale pmf
def pmf_rescale(PMF1):
    #PMF_new = []
    PMF_aux = [PMF1[i][0] for i in range(0, len(PMF1))] 
    PMF_new = [[j, 0] for j in range(0, 5000) if j not in PMF_aux] + PMF1
    PMF_new.sort()
    PMF_new = [PMF_new[i][1] for i in range(0, len(PMF_new))]
    return PMF_new

#hellinger distance
def H(p, q):
  # distance between p an d
  # p and q are np array probability distributions
    n = len(p)
    sum = 0.0
    for i in range(n):
        sum += (np.sqrt(p[i]) - np.sqrt(q[i]))**2
    result = (1.0 / np.sqrt(2.0)) * np.sqrt(sum)
    return result


In [ ]:
ll = []

In [ ]:
#REFIT

#from the five selected appliances, obtain the appliances that exist within the building 
#selected appliances
app_sel= ["television", "microwave", "kettle", "washing machine", "dish washer"]
#appliances within building
app_build = [LIST_APP.appliances[i].type['type'] for i in range(0, len(LIST_APP.appliances))]
#selected appliances within building
app_sel_build = [app_sel[j] for j in range(0, len(app_sel)) if app_sel[j] in app_build] 

def get_final_pmf(appliances):
    for k in range(0, len(appliances)):
        LOAD = appliances[k]
        #original PMF
        LOAD1_array  =  next(LIST_APP[LOAD].load(sample_period = 60))['2014-06-20':'2014-09-20'].values.flatten()
        LOAD1_array2 = [math.floor(LOAD1_array[i]) for i in range(0, len(LOAD1_array))]
        val, cnt = np.unique(LOAD1_array2, return_counts=True)
        PMF = np.column_stack((val, cnt/len(LOAD1_array2))).tolist()
        #rescale PMF
        vars()["PMF_"+str(LOAD)] = pmf_rescale(PMF)
        ll.append(vars()["PMF_"+str(LOAD)])
        
get_final_pmf(app_sel_build)        

In [ ]:
#obtain similarities between appliances using Hellinger distance
similarities = [[H(ll[j%len(app_sel_build)], ll[(j+1+l)%len(app_sel_build)]) for l in range(0, len(ll)-1)] for j in range(0, len(ll))]

#obtain the median, standard deviation and range of the similarity of power profiles for each appliance
median_std_ran = [[round(np.median(similarities[i]), 2), round(np.std(similarities[i]),2), round(np.max(similarities[i])-np.min(similarities[i]),2)] for i in range(0, len(similarities))]


# Case study 3


# Relationship between similarity of usage patterns and NILM performance

In [ ]:
from deep_nilmtk.utils.templates import ExperimentTemplate
from deep_nilmtk.disaggregator import NILMExperiment
from deep_nilmtk.models.pytorch import UNETNILM, BERT4NILM, DAE, SAED, WindowGRU
from deep_nilmtk.data.loader.pytorch import BERTDataset, GeneralDataLoader, bert_dataloader

In [ ]:
#define experiment template for BERT4NILM
DATA_PATH = 'REFIT.h5' 
EXPERIMENT_NAME = 'BERT_TVWMFKT_H12_v1' 
RESULTS_PATH = 'BERT_TVWMFKT_H12_v1' 

template = ExperimentTemplate(data_path=DATA_PATH,
                              template_name= 'NILM22_experiment',
                              list_appliances=["kettle", "television"],
                              list_baselines_backends=[], #no implemented baseline for BERT
                              in_sequence=121, #121 #based on a s2p architecture
                              out_sequence=1,
                              max_epochs=50)

bertnilm = NILMExperiment({
                "model_class": BERT4NILM,
                "loader_class": BERTDataset,
                "model_name": 'bert101',
                'backend':'pytorch',
                'in_size': 480,
                'out_size':480,
                'custom_preprocess':None,
                'feature_type':'mains',
                'input_norm':'z-norm',
                'target_norm':None,
                'seq_type':'seq2seq',
                'learning_rate':10e-5,
                'stride': 10,
                'max_nb_epochs': 50,
                'kfold':5,
                
                'cutoff':{ # necessary to define CUT-OFFS and THRESHOLDS for the appliance consumption
                      'aggregate': 6000,
                     'washing machine': 3000,
                      'dish washer': 3000,
                    'television': 3000,
                    'microwave':3000,
                    'kettle':3000, 
                    'fridge':3000,                     
                  },
                'threshold':{
                     'washing machine': 0,
                      'dish washer': 0,
                    'television': 0,
                    'microwave': 0,
                    'kettle': 0,
                    'fridge': 0,                     
                  },
                  'min_on':{
                     'washing machine': 0,
                      'dish washer': 0,
                          'television': 0,
                    'microwave': 0,
                    'kettle': 0,
                      'fridge': 0, 
                  },
                  'min_off':{
                     'washing machine': 0,
                      'dish washer': 0,
                'television': 0,
                    'microwave': 0,
                    'kettle': 0,
                      'fridge':0, 
                  },                          
               
})

template.extend_experiment({
    'bert101':bertnilm
})

template.run_template(EXPERIMENT_NAME,
                      RESULTS_PATH,
                      f'{RESULTS_PATH}/mlflow')

In [ ]:
LOAD = "dish washer"
list_buildings =  ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '13', '14', '15', '16', '17', '18', '19', '20']
res = []

for j in range(0, len(list_buildings)):
    aux1 = dataset.buildings[int(list_buildings[j])].elec
    list_app = [aux1.appliances[i].type['type'] for i in range(0, len(aux1.appliances))]
    if LOAD in list_app:
        LIST_APP = aux1
        LOAD1 =  next(LIST_APP[LOAD].load(sample_period = 60))['2014-06-20':'2014-09-20']
        #len(LOAD1.values.flatten())/sum(LOAD1.values.flatten())
        res.append(round((sum(LOAD1.values.flatten()))**2, 2))
        #res.append('{:0.2e}'.format((sum(LOAD1.values.flatten()))**2))
    else: 
        res.append(np.nan)

In [ ]:
sim_usg_REFIT_median = pd.DataFrame(
    {'TV': [12.16, 3.36, 10.17, 12.48, 16.95, 15.58, 12.35, 15.5, 14.65, 7.43, 2.08, 14.93, 7.37, 3.35, 18.66, 3.31, 8.04, 6.27],
     'MW': [np.nan, 0.84, 1.1, 0.71, 0.89, 0.93, np.nan, 1.05, 0.96, 1.18, 32.58/2, 0.87, np.nan, 0.42, 0.92, 0.57, 0.62, np.nan],
     'KT': [np.nan, 0.8, 0.82, 0.72, 1.28, 1.17, 0.92, 1.41, 0.78, np.nan, 1.72, np.nan, np.nan, 0.59, np.nan, 0.58, 0.7, np.nan],
     'WM': [6.57, 0.8, 0.93, 0.72, 1.4, 0.96, 0.69, 1.41, 0.81, 1.13, 1.23, 1.6, 4.04, 0.59, 0.85, 0.58, 0.66, 3.68],
     'DW': [6.2, 1.05, 1.04, np.nan, 1.12, 1.14, 0.92, np.nan, 1.01, 1.18, 1.47, 1.6, 3.86, np.nan, 0.92, np.nan, 0.76, 3.32]       
    }, 
      index = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '13', '15', '16', '17', '18', '19', '20', '21']      
    )

sim_usg_REFIT_std = pd.DataFrame(
    {'TV': [0.38, 0.37, 0.6, 0.57, 1.16, 0.99, 0.77, 1.16, 0.75, 0.86, 12.9, 0.98, 0.18, 0.09, 0.38, 0.32, 0.38, 0.37],
     'MW': [np.nan, 1.28, 4.22, 5.62, 7.04, 6.71, np.nan, 6.96, 5.82, 3.78, 1.11, 6.68, np.nan, 2.32, 8.66, 1.34, 3.14, np.nan],
     'KT': [np.nan, 1.31, 4.38, 5.69, 7.95, 6.81, 6.14, 7.26, 6.57, np.nan, 14.38, np.nan, np.nan, 2.3, np.nan, 1.44, 3.48, np.nan],
     'WM': [5.96, 1.06, 3.82, 5.01, 6.6, 6.09, 5.5, 5.8, 6.15, 3, 13.89, 6.2, 3.51, 2.2, 8.47, 1.03, 3.33, 2.96],
     'DW': [5.58, 0.74, 3.67, np.nan, 6.85, 5.69, 5.32, np.nan, 5.63, 3.06, 13.35, 7.25, 3.34, np.nan, 8.14, np.nan, 3.02, 2.58]       
    }, 
      index = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '13', '15', '16', '17', '18', '19', '20', '21']      
    )

sim_usg_REFIT_range = pd.DataFrame(
    {'TV': [0.75, 0.92, 1.4, 1.28, 2.95, 2.47, 1.75, 2.75, 1.97, 1.89, 30.08, 2.34, 0.36, 0.21, 0.94, 0.77, 0.92, 0.74],
     'MW': [np.nan, 3.18, 10.1, 12.06, 16.38, 15.88, np.nan, 15.07, 13.58, 8.04, 2.84, 14.27, np.nan, 4.93, 18.55, 2.93, 7.4, np.nan],
     'KT': [np.nan, 3.25, 10.3, 12.21, 18.74, 16.27, 13.13, 15.86, 15.25, np.nan, 33.86, np.nan, np.nan, 4.96, np.nan, 3.15, 8.26, np.nan],
     'WM': [11.91, 2.58, 8.97, 10.64, 15.48, 14.22, 11.68, 12.49, 14.31, 6.56, 32.4, 13.6, 7.03, 4.75, 18.12, 2.19, 7.77, 5.91],
     'DW': [11.16, 1.74, 8.61, np.nan, 16.06, 13.43, 11.4, np.nan, 13.13, 6.71, 31.26, 15.73, 6.67, np.nan, 17.3, np.nan, 7.01, 5.17]       
    }, 
      index = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '13', '15', '16', '17', '18', '19', '20', '21']      
    )

sim_pow_prof_REFIT_median = pd.DataFrame(
    {'TV': [0.36, 0.19, 0.39, 0.45, 0.47, 0.99, 0.37, 0.99, 0.99, 0.96, 0.23, 0.39, 0.35, 0.9, 0.99, 1, 0.99, 0.38],
     'MW': [np.nan, 0.16, 0.22, 0.63, 0.42, 0.98, np.nan, 0.99, 0.19, 0.76, 0.98, 0.62, np.nan, 0.98, 0.99, 0.13, 0.99, np.nan],
     'KT': [np.nan, 0.17, 0.22, 0.45, 0.4, 0.58, 0.21, 0.99, 0.2, np.nan, 0.23, np.nan, np.nan, 0.9, np.nan, 0.14, 0.56, np.nan],
     'WM': [0.23, 0.16, 0.21, 0.42, 0.62, 0.55, 0.23, 0.84, 0.22, 0.74, 0.28, 0.38, 0.26, 0.87, 0.98, 0.14, 0.54, 0.29],
     'DW': [0.23, 0.17, 0.26, np.nan, 0.43, 0.56, 0.23, np.nan, 0.2, 0.76, 0.22, 0.39, 0.29, np.nan, 0.99, np.nan, 0.55, 0.34]       
    }, 
      index = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '13', '15', '16', '17', '18', '19', '20', '21']      
    )

sim_pow_prof_REFIT_std = pd.DataFrame(
    {'TV': [0, 0.02, 0.02, 0.13, 0.09, 0, 0.02, 0, 0.02, 0.03, 0.33, 0.14, 0.03, 0.02, 0.01, 0.01, 0.01, 0.05],
     'MW': [np.nan, 0.04, 0.11, 0.04, 0.16, 0.02, np.nan, 0.07, 0.37, 0.32, 0.03, 0.03, np.nan, 0.03, 0.01, 0.42, 0.01, np.nan],
     'KT': [np.nan, 0.04, 0.11, 0.22, 0.16, 0.43, 0.08, 0.35, 0.36, np.nan, 0.35, np.nan, np.nan, 0.39, np.nan, 0.42, 0.44, np.nan],
     'WM': [0.13, 0.01, 0.08, 0.22, 0.02, 0.42, 0.08, 0.31, 0.31, 0.29, 0.28, 0.2, 0.06, 0.38, 0.42, 0.4, 0.43, 0.04],
     'DW': [0.13, 0.02, 0.08, np.nan, 0.1, 0.41, 0.06, np.nan, 0.34, 0.01, 0.35, 0.2, 0.09, np.nan, 0.42, np.nan, 0.43, 0.09]       
    }, 
      index = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '13', '15', '16', '17', '18', '19', '20', '21']      
    )

sim_pow_prof_REFIT_range = pd.DataFrame(
    {'TV': [0, 0.04, 0.05, 0.29, 0.24, 0.01, 0.04, 0.01, 0.06, 0.08, 0.79, 0.3, 0.06, 0.05, 0.01, 0.02, 0.03, 0.09],
     'MW': [np.nan, 0.1, 0.3, 0.09, 0.45, 0.04, np.nan, 0.15, 0.91, 0.76, 0.07, 0.06, np.nan, 0.06, 0.02, 0.93, 0.01, np.nan],
     'KT': [np.nan, 0.12, 0.31, 0.53, 0.45, 0.88, 0.18, 0.74, 0.91, np.nan, 0.87, np.nan, np.nan, 0.86, np.nan, 0.93, 0.88, np.nan],
     'WM': [0.26, 0.03, 0.21, 0.52, 0.05, 0.87, 0.18, 0.72, 0.73, 0.68, 0.65, 0.5, 0.13, 0.86, 0.89, 0.85, 0.87, 0.09],
     'DW': [0.26, 0.06, 0.19, np.nan, 0.24, 0.85, 0.13, np.nan, 0.81, 0.22, 0.87, 0.5, 0.18, np.nan, 0.9, np.nan, 0.87, 0.18]       
    }, 
      index = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '13', '15', '16', '17', '18', '19', '20', '21']      
    )

bert_perf_nde = pd.DataFrame(
    {'TV': [1, 1.06, 0.87, 0.84, 0.94, 0.81, 0.88, 0.94, 0.93, 0.51, 1.01, 0.86, 0.89, 1, 0.71, 0.78, 1.23, 0.98],
     'MW': [np.nan, 1.01, 1, 1.2, 1.1, 1.2, np.nan, 1.38, 1, 1.01, 0.89, 0.98, np.nan, 1.02, 1.04, 1.25, 0.95, np.nan],
     'KT': [np.nan, 0.67, 1.11, 0.69, 1, 0.8, 1.13, 0.83, 0.87, np.nan, 0.97, np.nan, np.nan, 0.73, np.nan, 0.73, 0.64, np.nan],
     'WM': [1.21, 1.06, 0.91, 0.70, 1.01, 0.94, 0.87, 0.99, 0.92, 0.85, 1.08, 0.66, 0.93, 1.01, 0.92, 0.92, 0.86, 1.03],
     'DW': [0.89, 0.76, 0.84, np.nan, 0.68, 0.65, 0.72, np.nan, 0.74, 0.92, 0.89, 1.12, 0.81, np.nan, 0.73, np.nan, 0.85, 0.84]       
    }, 
      index = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '13', '15', '16', '17', '18', '19', '20', '21']      
    )


bert_perf_totalcons = pd.DataFrame(
    {'TV': [1128248569399.0, 151045159696.56, 27480324503565.6, 10008100191736.93, 13062438739325.02,
            25650993776535.28, 1501405391527.58, 10368462691303.87, 14253725917044.38, 64638951170814.14,
            222702333746.61, 6655850717486.62, 20837124641123.23, 5322778964033.4, 25571032356076.18,
            2876215691419.9, 3783023192906.98, 1801438687723.52],
     
     'MW': [np.nan, 47525660353.46, 40480204228.49, 579164723024.76, 6005144671756.94, 549666309554.67, np.nan,
            1024082677668.77, 53165278669.33, 54393304859.59, 13766781060055.6, 243967631259.16, np.nan, 
            431044038574.19, 205444366131.6, 41903467982.7, 929516261453.86, np.nan],
     
     'KT': [np.nan, 7941428989181.46, 4515837943679.64, 1282901222030.71, 4159850216947.46, 7699499838343.82,
            1393572574729.52, 6120092199997.52, 9503645620207.67, np.nan, 5334416173150.34, np.nan, np.nan, 9506340995359.38,
            np.nan, 2379917691247.21, 2746099569934.27, np.nan],
     
     'WM': [1221521102641.88, 3214106238782.06, 11214951439571.72, 1878160219899.32, 17947144058589.97,
            465227275546.1, 12189039321254.93, 8296679684596.6, 3737228599854.77, 18058712988334.66, 7117071157425.71,
            3089938438742.84, 2449100133918.52, 1199404462635.13, 263227503306.23, 600303562814.26, 1076292988734.05, 
            1868590072912.27],
     
     'DW': [1701209104780.3, 29413839462868.58, 41065594497419.92, np.nan, 26031835470645.14, 2145999138762.29,
            41829069193706.96, np.nan, 31261822634716.58, 26456351453160.26, 5393365159318.93, 2514236.39, 4925568195944.7,
            np.nan, 5165897174189.15, np.nan, 2348060169516.46, 25555661661789.82]      
    }, 
      index = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '13', '15', '16', '17', '18', '19', '20', '21']      
    )


In [ ]:
mask = bert_perf_totalcons.isnull()
from matplotlib import rcParams
rcParams['figure.figsize'] = 7, 7               
ax = sns.heatmap(bert_perf_totalcons, annot=True, linewidth=.5, mask = mask, cmap="coolwarm", vmin=0, vmax=6.7e+13, fmt = '.1e')#fmt='.2f')
ax.set(xlabel='Appliance', ylabel='House', title = 'Total Squared Consumption REFIT')

In [ ]:
mask = bert_perf_totalcons.isnull()
from matplotlib import rcParams
rcParams['figure.figsize'] = 7, 7                  #PiYG
ax = sns.heatmap(sim_pow_prof_REFIT_std, annot=True, linewidth=.5, mask = mask, cmap="coolwarm", vmin=0, vmax=0.5, fmt = '.2f')#fmt='.2f')
ax.set(xlabel='Appliance', ylabel='House', title = 'Standard Deviation Overall Similarity Power Profiles REFIT')